In [1]:
from types import SimpleNamespace
%run -i GMD.py
%run -i iman_refine.ipynb
import pims
import re
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
%run -i BkVault.ipynb
import tifffile as ti

In [2]:
def get_input_image(p,op):
    #read the data by pims nd2 reader 
    if not p.in_series and p.in_series != 0:
        raise ValueError("Please specify input series")
    if not p.bg_series:
        raise ValueError("Please specify background series")
    if op['unmix']:
        raise ValueError("Perform linear channel unmixing is unavaliable")
    dao =  pims.bioformats.BioformatsReader(p.path, series = p.in_series)
    daobg =  pims.bioformats.BioformatsReader(p.path, series = p.bg_series)
    dao.bundle_axes = 'yxc'
    daobg.bundle_axes = 'yxc'
    bkf = BkVault(ip,op,daobg)
    nc = len(op['cind'])
    t = p.t
    xy = p.xy
    X =  bkf.flat(xy)
    im = np.zeros((dao.sizes['y'], dao.sizes['x'], nc,t))
    dao.iter_axes = 't'
    for i in range(p.t):
        #initial the blank array of image size
        ims = np.zeros((dao.sizes['y'], dao.sizes['x'], nc))
        for sc in range(nc):  # For each Channel,load image
            ims[:, :, sc] = np.array(dao[i][:,:,sc])
        #refine the image
        ims, e_inv = iman_refine(ims, ip['bval'], X)
        #get background value
        bkit, bkf = bkf.pull(daobg, p.xy, i, op['cind'], p.z)
        #subtract bg from image
        ims = ims - np.reshape(bkit, (1, 1, len(bkit)))
        im[:,:,:,i] = ims
    
    return im


In [3]:
p = SimpleNamespace(nclr = [0,0,1], 
                    cclr = [0,1,0.5], 
                    pastinfo = [], 
                    imthresh = 95, 
                    t = 5, #ie the number of time points you want to get. Max = 228
                    c = op['seg']['chan'][0], 
                    xy = 1, 
                    z = 1, 
                    display = True,
                    path = r"/Volumes/Extreme SSD/Living cell image/Original data and scripts/Project/20180123 MCF10 EKAR Fra T4 ratio expt downstairs.nd2",
                    in_series = 0,
                    bg_series = 60
                   )
GMD = mdi
# Adjust size parameters for calibration
pxscl = (GMD['cam']['PixSizeX'][0] + GMD['cam']['PixSizeY'][0]) / 2
op['seg']['minD'] = op['seg']['minD'] / pxscl
op['seg']['maxD'] = op['seg']['maxD'] / pxscl

In [4]:
im = get_input_image(p,op)
#TCYX
im1 = np.swapaxes(im,3,1)
im2 = np.swapaxes(im1,0,2)
im3 = np.swapaxes(im2,0,1)
im3 = im3.astype('float32')

URLError: <urlopen error [Errno 61] Connection refused>

In [5]:
im.shape,im3.shape

((1080, 1280, 4, 25), (25, 4, 1080, 1280))

In [6]:
#assign labels
labels_per_frame = mdi['exp']['Channel']
image_labels = labels_per_frame*p.t

In [7]:
#! is there a better way to save them as nd2 file in ImageJ?
ti.imwrite( 
     r'C:\Users\yuxin\Desktop\tc_data\pre_processed_images1.tif',
     im3,
     imagej=True,
     resolution=(1/pxscl, 1/pxscl),
     metadata={
         'spacing': 0,
         'unit': 'microns',
         'axes': 'TCYX',
         'Labels': image_labels,
         'Channel': {'Name': ['DAPI', 'EKAR_CFP','EKAR_YFP','Fra1_RFP']},
     }
)

# For later use

from json import JSONEncoder
import json

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
#Serialization
numpyData = {'im': im3}
print("serialize NumPy array into JSON and write into a file")
with open("numpyData.json", "w") as write_file:
    json.dump(numpyData, write_file, cls=NumpyArrayEncoder)

#TCYX == XYCT
im1 = np.swapaxes(im,3,1)
im2 = np.swapaxes(im1,0,2)
im3 = np.swapaxes(im2,0,1)

#XYCT == TCYX reshape the dims
im3 = np.swapaxes(im,0,1)

dao =  pims.bioformats.BioformatsReader(p.path, series = p.in_series)
dao.bundle_axes = 'yxc',
dao.iter_axes = 't'
dao.shape

In [9]:
#Linearly unmix color channel cross-talk —————— error